In [ ]:
import requests

In [ ]:
import http.client

conn = http.client.HTTPSConnection("openapi.emtmadrid.es")

payload = ""

headers = {
    'X-ClientId': "4614aca0-0a67-4bb9-b912-87b9bd62003c",
    'passKey': "6D42B608EFDB34D1EE2623846AF4CFB3B88161A28C6CF6973D9569FA746757BF907C058CA211FD8F5787A918E928F30D82F0EFFE8F219F4A54BF53BA10549F06"
    }

conn.request("GET", "/v2/mobilitylabs/user/login/", payload, headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

In [ ]:
# LOGIN REQUEST

url = "https://openapi.emtmadrid.es/v2/mobilitylabs/user/login/"

headers = {
    "X-ClientId": "4614aca0-0a67-4bb9-b912-87b9bd62003c",
    "passKey": "6D42B608EFDB34D1EE2623846AF4CFB3B88161A28C6CF6973D9569FA746757BF907C058CA211FD8F5787A918E928F30D82F0EFFE8F219F4A54BF53BA10549F06"
}

response = requests.request("GET", url, headers=headers)
access_token = response.json()["data"][0]["accessToken"]

In [ ]:
headers = {"accessToken": access_token}

url_bicimad_stations = "https://openapi.emtmadrid.es/v2/transport/bicimad/stations"
url_parking_details = "https://openapi.emtmadrid.es/v1/citymad/places/parking/{parking_id}/ES"
url_parkings_availability = "https://openapi.emtmadrid.es/v3/citymad/places/parkings/availability"
url_traffic = "https://informo.madrid.es/informo/tmadrid/pm.xml"

In [ ]:
response = requests.request("GET", url_bicimad_stations, headers=headers)
bicimad_stations = []
for record in response.json()["data"]:
    x = [record["id"],
         record["name"],
         record["geometry"]["coordinates"][0],
         record["geometry"]["coordinates"][1],
         record["free_bases"], 
         record["total_bases"],
         record["reservations_count"]
        ]
    bicimad_stations.append(x)

In [ ]:
response = requests.request("GET", url_parkings_availability, headers=headers)
parkings_availability = []
for record in response.json()["data"]:
    x = [record["id"], 
         record["freeParking"],
         record["geometry"]["coordinates"][0],
         record["geometry"]["coordinates"][1]
        ]
    parkings_availability.append(x)

In [ ]:
parkings = []
for parking_id, free_parking, _, _ in parkings_availability:
    
    if not free_parking:
        continue
        
    response = requests.request("GET", url_parking_details.format(parking_id=parking_id), headers=headers)
    for record in response.json()["data"]:
        x = [record["id"], 
             record["name"], 
             next((occ["content"] for occ in record["features"] if occ["name"] == "Total"), None)
            ]
        parkings.append(x)

In [ ]:
from xml.etree import ElementTree 

response = requests.request("GET", url_traffic)
root = ElementTree.fromstring(response.text)
all_data = zip(root.findall(".//pm/idelem"), 
               root.findall(".//pm/intensidad"),
               root.findall(".//pm/ocupacion"),
               root.findall(".//pm/carga"))
traffic_real_time = [[x.text,y.text,z.text,t.text] for x, y, z, t in all_data]

In [ ]:
output_dir = "/home/jovyan/notebooks/madflow/locations"
df = None
for i in range(25,231):
    try:
        url = f"https://datos.madrid.es/egob/catalogo/202468-{i}-intensidad-trafico.csv"
        dfi = pd.read_csv(url, delimiter=";", encoding="latin_1", 
                          usecols=["id", "nombre", "longitud", "latitud"])
        df = pd.concat([df, dfi]) if df is not None else dfi
    except Exception as e:
        pass

df["id"] = pd.to_numeric(df["id"], errors="coerce")
df["longitud"] = pd.to_numeric(df["longitud"], errors="coerce")
df["latitud"] = pd.to_numeric(df["latitud"], errors="coerce")
df = df.dropna(how="any")
df["id"] = df["id"].astype(int)
df.groupby("id").last().to_parquet("/home/jovyan/notebooks/madflow/locations.parquet")

In [ ]:
from bs4 import BeautifulSoup
import requests

list_items = "https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=ee941ce6ba6d3410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD"
response = requests.get(list_items)

soup = BeautifulSoup(response.text)
[x.get("href") for x in soup.find_all(class_="asociada-link ico-csv")]

In [1]:
from html.parser import HTMLParser
from urllib.request import urlopen

# Custom HTML parser to extract 'href' attributes from elements with a specific class
class LinkExtractor(HTMLParser):
    url = "https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=ee941ce6ba6d3410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD"

    def __init__(self):
        super().__init__()
        self.links = []
        with urlopen(self.url) as response:
            html_content = response.read().decode('utf-8')
            self.feed(html_content)


    def handle_starttag(self, tag, attrs):
        # Check for 'a' tags and attributes
        if tag == 'a':
            attr_dict = dict(attrs)
            if "class" in attr_dict and "asociada-link ico-csv" in attr_dict["class"]:
                href = attr_dict.get("href")
                if href:
                    self.links.append(href)

# Initialize parser and feed it the HTML content
parser = LinkExtractor()
href_links = parser.links
href_links

['/egob/catalogo/202468-230-intensidad-trafico.csv',
 '/egob/catalogo/202468-227-intensidad-trafico.csv',
 '/egob/catalogo/202468-224-intensidad-trafico.csv',
 '/egob/catalogo/202468-221-intensidad-trafico.csv',
 '/egob/catalogo/202468-218-intensidad-trafico.csv',
 '/egob/catalogo/202468-215-intensidad-trafico.csv',
 '/egob/catalogo/202468-212-intensidad-trafico.csv',
 '/egob/catalogo/202468-209-intensidad-trafico.csv',
 '/egob/catalogo/202468-206-intensidad-trafico.csv',
 '/egob/catalogo/202468-203-intensidad-trafico.csv',
 '/egob/catalogo/202468-200-intensidad-trafico.csv',
 '/egob/catalogo/202468-197-intensidad-trafico.csv',
 '/egob/catalogo/202468-194-intensidad-trafico.csv',
 '/egob/catalogo/202468-191-intensidad-trafico.csv',
 '/egob/catalogo/202468-188-intensidad-trafico.csv',
 '/egob/catalogo/202468-185-intensidad-trafico.csv',
 '/egob/catalogo/202468-182-intensidad-trafico.csv',
 '/egob/catalogo/202468-179-intensidad-trafico.csv',
 '/egob/catalogo/202468-176-intensidad-trafico

In [ ]:
url = "https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=ee941ce6ba6d3410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD"

with urlopen(url) as response:
    html_content = response.read().decode('utf-8')

In [ ]:
import requests
url = "https://openapi.emtmadrid.es/v1/citymad/places/arroundxy/ES/"

payload = {
    "coordinates": {
        "longitude": -3.7004,
        "latitude": 40.41465,
        "radius": 20000
    },
    "family": [{"familyCode": "101"}]
}
headers = {
    "Content-Type": "application/json",
    "accessToken": access_token
}

points_of_interest = []
response = requests.request("POST", url, json=payload, headers=headers)

raw_poi = next((source for source in response.json()["data"] if source["sourceName"] == 'Point of Interest from MADRID-DESTINO'), {}).get("data", [])
output = [[record["category"], record["longitude"], record["latitude"], record["idPoi"], record["name"]] for record in raw_poi]

In [ ]:
len(output)

In [ ]:
import json
import http.client

conn = http.client.HTTPSConnection("openapi.emtmadrid.es")

payload = '{"coordinates":{"longitude":-3.7004000,"latitude":40.4146500,"radius":10000},\n"family":[{"familyCode":"101"}]}'

headers = {
    'Content-Type': "application/json",
    'accessToken': access_token
    }

conn.request("POST", "/v1/citymad/places/arroundxy/ES/", payload, headers)

res = conn.getresponse()
response_dict = json.loads(res.read().decode("utf-8"))
raw_poi = next((source for source in response_dict["data"] if source["sourceName"] == 'Point of Interest from MADRID-DESTINO'), {}).get("data", [])
output = [[record["category"], record["longitude"], record["latitude"], record["idPoi"], record["name"]] for record in raw_poi]

In [ ]:
output